<a target="_blank" href="https://colab.research.google.com/github/visze/foundations_in_medicine/blob/master/biomedical_databases/tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🧬 Tutorial Biomedical Databases

## 🧬 Background

Several genes are linked to hereditary cancer, including **BRCA1**, **BRCA2**, and **TP53**. These genes play a crucial role in the body's ability to repair DNA damage and regulate cell growth. 🔬 Mutations in these genes can lead to an increased risk of developing certain types of cancer, particularly breast and ovarian cancer.

### ❓Question


> **💡 Look up the gene TERT in various databases. What is its function, and what diseases are associated with mutations in this gene?**


Besides coding mutations, regulatory mutations can also impact gene function. Gene coding regions constitute only around 2% of the human genome, while regulatory regions make up a much larger portion (up to 15-20% depending on the source). Their main purpose is to regulate the expression of genes. 🧫 Every cell has the same sequence of DNA, but their purpose, shape, etc., differ. We can define two different aspects of regulation:

1. **🗺️ Spatial regulation:**
   - Organs
   - Cell types
   - Single cells (e.g., different immune cells)
2. **⏱️ Temporal regulation:**
   - Cell cycle phase
   - Differentiation phase
   - Developmental stage
   - Induced transcription (environmental stimuli)

The regulation of transcription is controlled by the binding of proteins or RNA to regulatory nucleic acid sequences. These sequences are often classified into **promoters** and **enhancers**.

**🚦 Promoters** are typically located directly upstream of the gene they regulate and serve as the binding site for RNA polymerase and other transcription factors necessary for initiating transcription.

**⚡ Enhancers**, on the other hand, can be located far away from the gene they regulate (upstream, downstream, or even within introns) and function to increase the transcriptional activity of a gene by providing additional binding sites for transcription factors.

![Regulation](resources/picture/regulation.png)

Mutations in regulatory regions can affect gene expression levels, leading to diseases such as cancer. 🦠 TERT promoter mutations have been identified in various cancers, including melanoma, glioblastoma, and bladder cancer. These mutations can lead to increased TERT expression, which may contribute to the uncontrolled growth of cancer cells. 📈

## 🔍 Variants in the TERT promoter region

Now we will look up different databases to find information about variants in the TERT promoter region. 📚 We'll try to solve this question "programmatically"—this means using code to query the databases instead of using their web interfaces (when it makes sense). 💻

### ❓Question

> **💡 Find out the genomic coordinates (GRCh38) of the TERT promoter region.**

📌 Helpful resources:
- [UCSC Genome Browser](https://genome.ucsc.edu/) 🌐
- [ENCODE SCREEN database](https://screen.wenglab.org). SCREEN is a database derived from ENCODE data. They annotate candidate cis-regulatory elements (cCREs) in the human and mouse genomes based on a large number of functional genomics experiments. 🧪

### 🌍 Population variants from gnomAD

The overall question we want to solve is: are their variants in the TERT promoter region in our human population that can lead to misregulation of TERT? To answer this question, we will use the [gnomAD database](https://gnomad.broadinstitute.org/). 🧬 gnomAD (Genome Aggregation Database) is a comprehensive resource that aggregates and harmonizes exome and genome sequencing data from a wide variety of large-scale sequencing projects. It provides allele frequency data for genetic variants across diverse populations, making it a valuable tool for researchers studying human genetics and disease. 🧫

GnomAD uses [GraphQL](https://graphql.org/) to provide programmatic access to their database. 🔌 GraphQL is an alternative to REST APIs. Instead of multiple endpoints, there is only one endpoint. The user defines the structure of the response data in the request. ✨

First we need to install the GQL GraphQL client library for Python. 📦

In [ ]:
%%bash
pip install gql

Starting with GraphQL is not that easy. 🤔 But many databases use it, and so it might be hepful to try it out and understand the concept. GnomAD has a (very short) documentation here: https://gnomad.broadinstitute.org/data#api.

However, it only shows you the `gene` query. But there is more! 🎯 You have `region`, `transcript`, `variant`, and many more searches. You can find them out by clicking on this link: https://gnomad.broadinstitute.org/api? and click yourself through the panel on the right side on `QQuery` and further.

Also it is important to understand what the query returns because you have to specifically add this to your query. E.g. the `region` query returns a `Region` object. You can find out what fields are available for this object by clicking on the `Region` link in the panel on the right side. 📋

### ❓ TODO

> **💡 Insert your region in the query by setting the variable `chrom`, `start` and `stop`**

In [ ]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

transport = AIOHTTPTransport(url="https://gnomad.broadinstitute.org/api")
client = Client(transport=transport, fetch_schema_from_transport=True)

# TODO DEFINE YOUR TERT PROMOTER REGION HERE
# Define region variables
chrom = "5" # like "1"
start = 1294944 # like 1000000
stop = 1295294 # like 1010000

query = gql(
  f"""
  query RegionQuery {{
    region(chrom: "{chrom}", start: {start}, stop: {stop}, reference_genome: GRCh38) {{
    variants(dataset: gnomad_r4) {{
      variant_id
      chrom
      pos
      ref
      alt
      joint {{
      ac
      an
      }}
    }}
    }}
  }}
  """
)
result = await client.execute_async(query)
print(result)

You see we get a JSON string back of our results. It is very easy to create a pandas DataFrame out of it. 🐼 We only have to care of nested objects (which is only the `joint` field).

In [ ]:
import pandas as pd

# Extract variants data from the result dictionary
variants_data = result['region']['variants']

# Create a pandas DataFrame
df_gnomad_variants = pd.DataFrame(variants_data)

# Expand the 'joint' column into separate 'ac' and 'an' columns
df_gnomad_variants['ac'] = df_gnomad_variants['joint'].apply(lambda x: x['ac'])
df_gnomad_variants['an'] = df_gnomad_variants['joint'].apply(lambda x: x['an'])

# Drop the original 'joint' column
df_gnomad_variants = df_gnomad_variants.drop(columns=['joint'])

# Display the DataFrame
df_gnomad_variants

### ❓ Question

> **💡 How many variants did you find?**
>
> **💡 What is the most common variant?**

In [ ]:
# TODO ADD YOUR CODE HERE

In [ ]:
# --- EXAMPLE RESULT ---
# Calculate allele frequency
df_gnomad_variants["af"] = df_gnomad_variants["ac"] / df_gnomad_variants["an"]

# Find the most common variant
df_gnomad_variants.sort_values(by="af", ascending=False).head(1)

### 🏥 Clinically annotated variants from ClinVar

Now we want to find out whoich variants are already been annotated for any clinical significance (no matter if related to cancer or not). For this, we will use the [ClinVar database](https://www.ncbi.nlm.nih.gov/clinvar/). 🏥 ClinVar is a publicly accessible database that aggregates information about genomic variations and their relationship to human health. It is maintained by the National Center for Biotechnology Information (NCBI), a part of the U.S. National Library of Medicine. 📚

ClinVar usually provides downloadable files for bulk access to their data. 📥 Usually, people use these files. ClinVar has an API for submission of new data and also for retrieval. However, ClinVar data is also available through the gnomAD API (maybe not the most recent one). But we already used gnomAD successfully, so we'll continue here with gnomAD to avoid learning any new API. 🎓

ClinVar data is available through the `clinvar_variant` field in the gnomAD API. You can query by variant ID or search within a region. 🔎

In [ ]:
# Query ClinVar variants within a region
clinvar_region_query = gql(
    f"""
    query ClinVarRegionQuery {{
      region(chrom: "{chrom}", start: {start}, stop: {stop}, reference_genome: GRCh38) {{
        clinvar_variants {{
          variant_id
          chrom
          pos
          ref
          alt
          clinical_significance
        }}
      }}
    }}
    """
)

clinvar_region_result = await client.execute_async(clinvar_region_query)
print("\nClinVar Region Query Result:")
print(clinvar_region_result)

Let's create a pandas dataframe out of it again. 🐼

In [ ]:

# Extract variants data from the result dictionary
clinvar_variants_data = clinvar_region_result['region']['clinvar_variants']

# Create a pandas DataFrame
df_clinvar_variants = pd.DataFrame(clinvar_variants_data)

# Display the DataFrame
df_clinvar_variants

### ❓ Question

> **💡 How many ClinVar variants did you find?**
>
> **💡 Are all of them pathogenic? What is their clinical significance?**

In [ ]:
# TODO ADD YOUR CODE HERE

In [ ]:
# --- EXAMPLE RESULT ---
# Count clinical significance categories
df_clinvar_variants['clinical_significance'].value_counts()

### 🔗 Annotate population variants with ClinVar data

Merge gnomAD variants with ClinVar variants to find clinical significance of variants in the region. 🧩

In [ ]:
# Merge gnomAD variants with ClinVar variants
# Create a common key for merging based on variant_id, chromosome, position, ref, and alt
df_merged_variants = df_gnomad_variants.merge(
    df_clinvar_variants,
    left_on=['variant_id', 'chrom', 'pos', 'ref', 'alt'],
    right_on=['variant_id', 'chrom', 'pos', 'ref', 'alt'],
    how='left'
)

# Display the merged DataFrame
df_merged_variants

### ❓ Question

> **💡 What clinical significance do the top 5 variants with the highest allele frequency have?**
>
> **💡 What is the allele frequency or allele count of pathogenic variants?**

In [ ]:
# TODO ADD YOUR CODE HERE

In [ ]:
# --- EXAMPLE RESULT ---
df_merged_variants[df_merged_variants['clinical_significance'].notna()].sort_values(by="af", ascending=False).head(5)[['variant_id', 'af', 'clinical_significance']]

In [ ]:
# --- EXAMPLE RESULT ---
df_merged_variants[df_merged_variants['clinical_significance'] == 'Pathogenic'][['variant_id', 'af', 'ac', 'clinical_significance']]

## 🎯 Linking Motifs to Variant Effects

We know that regulation is controlled by the binding of proteins or RNA to regulatory nucleic acid sequences. 🧬 The proteins are called **transcription factors (TFs)**. TFs bind to specific DNA sequences, often referred to as **motifs**. 🔑 Mutations in these motifs can disrupt the binding of TFs, but also create new binding sites, leading to changes in gene expression. The direction of the expression change depends on whether the binding TF is an activator or repressor of transcription. ⚖️

Here is a visualization of an example motif. You can see which bases are preferred at which position in the motif. The higher the letter, the more often this base is found at this position in known binding sites for this TF. Usually motifs are storred as position weight matrices (PWMs) or position frequency matrices (PFMs).

![example motif](https://hocomoco13.autosome.org/final_bundle/hocomoco13/H13CORE/logo_large/ALX1.H13CORE.0.SM.B_direct.png)

Keep in mind that binding of TFs is highly tissue-specific. 🎭 Not every TF is present in every cell type. Therefore, mutations in motifs may have different effects depending on the cellular context. Also, we will find many more motifs than actual binding events. Just because a motif is present does not mean that the TF actually binds there in a given cell type. ⚠️

We will now analyze our variants from our previous analysis in the context of motifs. 🔬 We will use all variants that have an entry in ClinVar and will link them to creation/disruption of motifs. Therefore, we use a tool called [FIMO](https://meme-suite.org/meme/tools/fimo) 🛠️ and a motif database called [HOCOMOCO](https://hocomoco13.autosome.org/). 📊

### 🚀 Getting Started

We will now run FIMO from the MEME suite in a container. 🐳 It can be Docker, Apptainer, or in Colab we will use uDocker. Therefore, we will need some additional software and data files. 📂

**NOTE:** You might want to collapse the code cells of the "Getting Started" section (by clicking the "arrow down" button left of the "Getting Started" section) and click on the appearing button (▶️) to run all the 6 cells at once because they take some time (approx. 3-5 minutes).

- We will use the `IPython` package to beautify the notebook by removing the output of some cells. More information about it can be found [here](https://ipython.org/install.html).

In [ ]:
from IPython.display import clear_output

tutorial_container = "memesuite/memesuite:latest"
docker = False
apptainer = False

docker = !docker --help &>/dev/null; if [ $? -eq 0 ]; then echo 1; else echo 0; fi
docker = bool(int(docker[0]))
apptainer = !apptainer --help &>/dev/null; if [ $? -eq 0 ]; then echo 1; else echo 0; fi
apptainer = bool(int(apptainer[0]))

if docker:
    print("Docker is installed! We will use Docker to run FIMO")
elif apptainer:
    print("Apptainer is installed! We will use Apptainer to run FIMO")
else:
    print("Neither Docker nor Apptainer is installed. We assume you run the tutorial on Colab and we will install uDocker for Colab")

In [ ]:
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

if not docker and not apptainer:
    udocker = udocker_init()

In [ ]:
def runContainer(command):
    user_prompt = "\033[1;32muser@pc\033[0m"
    if docker:
        print(f"{user_prompt}$ docker run -v=${{PWD}}/resources:/data/run/resources:ro -v=${{PWD}}/results:/data/run/results:rw -u $(id -u):$(id -g) --workdir /data/run {tutorial_container} {command}")
        !docker run -v=${{PWD}}/resources:/data/run/resources:ro -v=${{PWD}}/results:/data/run/results:rw -u $(id -u):$(id -g) --workdir /data/run {tutorial_container} {command}
    elif apptainer:
        print(f"{user_prompt}$ apptainer run -B=${{PWD}}/resources:/data/run/resources -B=${{PWD}}/results:/data/run/results --cwd /data/run memesuite.sif {command}")
        !apptainer run -B=${{PWD}}/resources:/data/run/resources -B=${{PWD}}/results:/data/run/results --cwd /data/run memesuite.sif {command}
    else:
        !chmod o+rw results
        udocker(f"run -v=${{PWD}}:/data/run --workdir /data/run memesuite {command}")

In [ ]:
if docker:
    print(f"docker pull {tutorial_container}")
    !docker pull "{tutorial_container}"
    print(f"docker create --name=memesuite {tutorial_container}")
    !docker create --name=memesuite "{tutorial_container}" > /dev/null
elif apptainer:
    print(f"apptainer pull memesuite.sif docker://{tutorial_container}")
    !apptainer pull memesuite.sif docker://{tutorial_container} > /dev/null
else:
    udocker(f"pull {tutorial_container}")
    udocker(f"create --name=memesuite {tutorial_container}")
    clear_output()


In [ ]:
%%bash

mkdir -p results
mkdir -p resources
mkdir -p resources/hocomoco_v13

if [ ! -f "resources/hocomoco_v13/H13CORE_meme_format.meme.gz" ]; then
    wget -q https://raw.githubusercontent.com/kircherlab/ISMB-2025_IGVF-MPRA-Tutorial/main/06_variant_effects_and_motifs/resources/hocomoco_v13/H13CORE_meme_format.meme.gz -O resources/hocomoco_v13/H13CORE_meme_format.meme.gz
else
    echo "File 'H13CORE_meme_format.meme.gz' already exists. Skipping..."
fi

if [ ! -f resources/hocomoco_v13/H13CORE_meme_format.meme ]; then
  gzip -dc resources/hocomoco_v13/H13CORE_meme_format.meme.gz > resources/hocomoco_v13/H13CORE_meme_format.meme
else
    echo "File 'resources/hocomoco_v13/H13CORE_meme_format.meme' already exists. Skipping..."
fi


Now we install the required Python libraries. 📦✨

In [ ]:
# Install dependencies
!pip install biopython seaborn pandas numpy matplotlib

# clear cell output
clear_output()

---

### 🔍 Motif Identification from Sequence

Let's explore how to identify transcription factor binding motifs in DNA sequences. 🧬✨

#### 🗄️ TFBS Databases: JASPAR & HOCOMOCO

Transcription factors (TFs) regulate gene expression by binding to specific DNA motifs. 🔗 Identifying these motifs is crucial for understanding gene regulation, cell differentiation, and disease. 🎯

**Key Databases:**
- [JASPAR](https://jaspar.elixir.no/) 📚
- [HOCOMOCO](https://hocomoco13.autosome.org/) 📊

Both provide position weight matrices (PWMs) for TFBSs.  
> **⚠️ Note:**  
> Genome-wide PWM scanning can yield many false positives due to short motif lengths and genome size.

Here, we'll focus on **HOCOMOCO v13 (H13)** in .meme format. 📄

- **🏆 HOCOMOCO v13:**  
  - Curated for human and mouse TFBSs 🐭
  - ~1,200 motifs for 1,000 TFs
  - Includes ChIP-seq and computational predictions 🧪
  - Quality control metrics (e.g., supporting sequences, conservation) ✅
  - [Download HOCOMOCO v13](https://hocomoco13.autosome.org/downloads_v13) 📥

![HOCOMOCOv13 Website Screenshot](resources/picture/hocomocov13_website.png) 🖼️

#### 🛠️ TFBS Identification Tools

Several tools predict TFBSs using PWMs: 🔧

- **FIMO** (Find Individual Motif Occurrences, MEME suite) 🎯
- **HOMER** (Hypergeometric Optimization of Motif EnRichment) 📊
- ...and more! ✨

We'll use **FIMO** for this tutorial. 🚀

**📋 FIMO Usage:**

- Input: TFBS database (.meme format) + FASTA sequences 📄
- Output: Table of motif matches (`motif_id`, sequence name, start/end, matched sequence, etc.) 📊

We'll use the `.tsv` output for downstream analysis. 🔬

So we first need to prepare our input files for FIMO. 📝 We want to use all our variants that have an entry in ClinVar. Also, we only look (for simplicity) at single nucleotide variants (SNVs). 🧬

In [ ]:
df_clinvar_snvs = df_merged_variants[df_merged_variants['clinical_significance'].notna()]
df_clinvar_snvs = df_clinvar_snvs[df_clinvar_snvs['alt'].str.len() == 1]
df_clinvar_snvs = df_clinvar_snvs[df_clinvar_snvs['ref'].str.len() == 1]
df_clinvar_snvs

Now we need the FASTA sequence around the variant positions to run FIMO. 🧬 This can be done in various ways:
1. Copy and paste the sequence from a genome browser (the non-bioinformatics way) 📋
2. Use a tool like `bedtools` to extract the sequences programmatically. This requires the human reference genome to be available locally 🗄️
3. Use a Python library like Biopython or `pyfaidx` to fetch the sequences from a local file 🐍
4. Use an online service like the UCSC Table Browser or Ensembl REST API to fetch sequences 🌐

Because we already started with APIs, we will use the UCSC REST API to fetch the sequences. 🚀

In [ ]:
import requests

# Create a FASTA file with sequences for variants in df_clinvar_snvs
extend_around_variant = 10
fasta_lines = []

for _, row in df_clinvar_snvs.iterrows():
    variant_id = row['variant_id']
    chrom = row['chrom'].replace('chr', '')
    pos = row['pos']
    ref = row['ref']
    alt = row['alt']
    
    # Get sequence context (20bp window around variant)
    seq_params = {
        "genome": "hg38",
        "chrom": f"chr{chrom}",
        "start": pos - extend_around_variant - 1,
        "end": pos + extend_around_variant
    }
    seq_r = requests.get("https://api.genome.ucsc.edu/getData/sequence", params=seq_params, timeout=30)
    seq_r.raise_for_status()
    ref_seq = seq_r.json()["dna"]

    # Create alt sequence by substituting the variant
    alt_seq = ref_seq[:extend_around_variant] + alt + ref_seq[extend_around_variant + len(ref):]
    
    # Add ref_seq and alt_seq to the DataFrame row
    df_clinvar_snvs.at[row.name, 'variant_pos'] = extend_around_variant + 1
    df_clinvar_snvs.at[row.name, 'ref_sequence'] = ref_seq
    df_clinvar_snvs.at[row.name, 'alt_sequence'] = alt_seq
    
    fasta_lines.append(f">REF_{variant_id}\n{ref_seq}")
    fasta_lines.append(f">ALT_{variant_id}\n{alt_seq}")

    df_clinvar_snvs.at[row.name, 'REF'] = f"REF_{variant_id}"
    df_clinvar_snvs.at[row.name, 'ALT'] = f"ALT_{variant_id}"

# Write to FASTA file
with open("resources/clinvar_variants_fimo_input.fa", "w") as f:
    f.write("\n".join(fasta_lines))

In [ ]:
# Display the DataFrame with sequences
df_clinvar_snvs[['variant_id', 'ref_sequence', 'alt_sequence']]

Our input file for FIMO looks like this: 📄✨

In [ ]:
%%bash
cat resources/clinvar_variants_fimo_input.fa | head

Let's run FIMO on the recently created input file: 🚀💻

In [ ]:
runContainer("fimo --o results/finding_tfbs_in_variants_H13 resources/hocomoco_v13/H13CORE_meme_format.meme resources/clinvar_variants_fimo_input.fa")

Finally, we load the output file and merge it with our variant metadata we used as input (for both alternative and reference sequences) to create a unified, filterable table. 📊🔗

In [ ]:
import os

# Check out FIMO output
fimo_input = "results/finding_tfbs_in_variants_H13"
fimo_result = os.path.join(fimo_input, 'fimo.tsv')
fimo_result_df = pd.read_csv(fimo_result, sep="\t", comment='#')

##### 📊 FIMO Output Columns

| Column           | Description                                      |
|------------------|--------------------------------------------------|
| `motif_id`       | ID of the matched motif 🏷️                       |
| `sequence_name`  | Name of the sequence with the motif 📝           |
| `start`/`stop`   | Start/stop positions of the motif match 📍       |
| `strand`         | Strand (+/-) ➡️                                  |
| `matched_sequence`| Sequence matching the motif 🧬                  |
| `p-value`        | Motif match p-value (default threshold: 1e-4) 📉 |
| `q-value`        | FDR-corrected p-value (higher confidence = lower) ✅ |

> **💡 Tip:** Lower q-values indicate higher confidence in motif matches.

In [ ]:
# Preview the FIMO result table
fimo_result_df.sort_values(by="q-value")

##### ❓ Question

> **💡 What is the average number of motifs found in our 21bp sequence?**
>
> **💡 Are all of them true TF binders?**
> 
> **💡 Do all motifs overlap our variant?**

In [ ]:
# TODO ADD YOUR CODE HERE

In [ ]:
# --- EXAMPLE RESULT ---
# Get statistics for the number of motifs found per sequence
sequence_counts = fimo_result_df['sequence_name'].value_counts()

print(f"Minimum motifs per sequence: {sequence_counts.min()}")
print(f"Mean motifs per sequence: {sequence_counts.mean():.2f}")
print(f"Maximum motifs per sequence: {sequence_counts.max()}")

##### 🔗 Combine FIMO Output with Variant Information

Let's merge the FIMO results with variant metadata for both reference and alternative alleles. 🧩

In [ ]:
important_cols_for_matching = ['variant_id', 'REF', 'ALT', 'ref_sequence', 'alt_sequence', 'variant_pos']

# Merge FIMO with variants based on alternative and reference sequence names
fimo_alt = fimo_result_df.merge(df_clinvar_snvs[important_cols_for_matching], left_on="sequence_name", right_on="ALT", how="inner")
fimo_ref = fimo_result_df.merge(df_clinvar_snvs[important_cols_for_matching], left_on="sequence_name", right_on="REF", how="inner")

# Rename columns to distinguish ALT and REF q-values
fimo_alt = fimo_alt.rename(columns={"q-value": "ALT_q-value", "sequence_name": "ALT_name"})
fimo_ref = fimo_ref.rename(columns={"q-value": "REF_q-value", "sequence_name": "REF_name"})

# Merge ALT and REF results on ID, motif_id, start, stop, strand
combined_fimo_df = pd.merge(
    fimo_alt[['variant_id', 'motif_id', 'start', 'stop', 'strand', 'ALT_name', 'ALT_q-value']],
    fimo_ref[['variant_id', 'motif_id', 'start', 'stop', 'strand', 'REF_name', 'REF_q-value']],
    on=['variant_id', 'motif_id', 'start', 'stop', 'strand'],
    how='outer'  # Ensure we include all motif occurrences
)

combined_fimo_df

We want to look into the variant-overlapping TFBS, which requires the information of the variant position. 📍 This info can be added from the variant table. ➕

In [ ]:
clinvar_variant_infos = ['variant_id', 'variant_pos', 'ref_sequence', 'alt_sequence']
variant_metadata_fimo_combined_df = combined_fimo_df.merge(df_clinvar_snvs, on="variant_id", how="inner")

variant_metadata_fimo_combined_df['variant_pos'] = variant_metadata_fimo_combined_df['variant_pos'].astype(int)
variant_metadata_fimo_combined_df

Now we have to make sure that the motif overlaps our variant position. 🎯 Therefore, we filter the combined table for motif start ≤ variant position ≤ motif end. ✅

In [ ]:
def motif_overlaps_variant(row, fimo_start_col='start', fimo_stop_col='stop', col_variant_pos='variant_pos'):
    """
    Iterate over each variant position and return True if the start/stop coordinates
    overlap the variant position.
    
    NOTE: It is important that we use 1-based coordinates for this comparison as FIMO uses 1-based coordinates.
    """
    variant_pos = row[col_variant_pos]
    if (int(row[fimo_start_col]) <= variant_pos and int(row[fimo_stop_col]) >= variant_pos):
        return True
    return False

variant_metadata_fimo_combined_df["is_variant_overlapping"] = variant_metadata_fimo_combined_df.apply(
    lambda row: motif_overlaps_variant(row, fimo_start_col='start', fimo_stop_col='stop', col_variant_pos='variant_pos'),
    axis=1
)

> **✅ Result:**  
> Now we have a table of significant variants and their overlapping TFBS predictions. Let's explore the numbers! 🔢📊

In [ ]:
# Preview combined variant-TFBS table
variant_metadata_fimo_combined_df[["variant_id", "motif_id", "is_variant_overlapping"]]

In [ ]:
# Filter for variant-overlapping TFBS
variant_overlapping_tfbs = variant_metadata_fimo_combined_df.loc[variant_metadata_fimo_combined_df['is_variant_overlapping']].copy()

In [ ]:
print("Number of Motif predictions within the given sequences", variant_overlapping_tfbs.shape[0])
print("Number of different variants we found at least one TFBS", variant_overlapping_tfbs['variant_id'].nunique())

### ⚖️ Motif Gain or Loss? Assessing TFBS Changes

We can compare if the motifs fit the reference or the alternative sequence better, so we know if the motif is more likely to bind at a specific allele. 🔬 But from just computational predictions we cannot tell if the motif is now bound or not bound anymore because of the SNV. However, we can increase the confidence of our predictions, which are prone to false positives. ✅

> **🔍 Observation:**  
> For 26 variants, we see 495 TFBS hits! 🎯  
> - Short motifs → many chance matches (false positives) ⚠️
> - Multiple TFBS can overlap the same variant 🔄
>
> **🤔 How to prioritize?**
> - Use more stringent q-value thresholds 📊
> - Focus on TFs expressed in your cell type 🧫
> - Use additional data (e.g., ChIP-seq) 🧪

#### 🎚️ Filter by q-value (Stringency)

Let's filter for motif matches with q-value < 0.001 for higher confidence. ✅

0.001 is very stringent. 📏 You can play around with this value to see how many motif matches you get with different thresholds. 🔧

In [ ]:
q_value_threshold = 0.001

variant_metadata_fimo_combined_overlap_filtered = variant_overlapping_tfbs.loc[
    (variant_overlapping_tfbs['ALT_q-value'] < q_value_threshold) | (variant_overlapping_tfbs['ALT_q-value'].isna())
].copy()

variant_metadata_fimo_combined_overlap_filtered = variant_metadata_fimo_combined_overlap_filtered.loc[
    (variant_metadata_fimo_combined_overlap_filtered['REF_q-value'] < q_value_threshold) | 
    (variant_metadata_fimo_combined_overlap_filtered['REF_q-value'].isna())
].copy()

In [ ]:
print("Number of Motif predictions within the given sequences", variant_metadata_fimo_combined_overlap_filtered.shape[0])
print("Number of different variants we found at least one TFBS", variant_metadata_fimo_combined_overlap_filtered['variant_id'].nunique())

In [ ]:
# Preview filtered motif-variant table
variant_metadata_fimo_combined_overlap_filtered

#### 🤔 Which TFBS Fits Better?

The q-value ranks motif matches, but longer motifs tend to have lower p/q-values. 📊 That's why a very low threshold gives us always motifs of lengths around 20. Let's use a **bitscore** (normalized for motif length) to compare motif fits near the variant. ⚖️

![multiple_tfbs_fitting_variant_example_long_vs_short.png](resources/picture/multiple_tfbs_fitting_variant_example_long_vs_short.png) 🖼️

##### 👁️ Which TFBS fits better by eye?

But by eye, I would say that the bottom one matches more important nucleotides. 🎯 Note that it is known that the p- and q-values are not reliable when comparing motifs with different lengths, because longer motifs get a lower p-value more easily. However, we can use a **bitscore** 📊 which is measured with a unified size around the variant position to make motifs of different sizes comparable. ⚖️

In [ ]:
from Bio import motifs

# Function used to compute the local bitscore
def extract_motif_names(meme_file):
    motif_names = []
    with open(meme_file, "r") as f:
        for line in f:
            if line.startswith("MOTIF"):
                motif_name = line.split(" ", 1)[1].strip()
                motif_names.append(motif_name)
    return motif_names


# Read the used motif file and extract the motifs
meme_file = "resources/hocomoco_v13/H13CORE_meme_format.meme"
motif_names = extract_motif_names(meme_file)

# Add the name while loading the file using Biopython
hocomoco_motif_dict = {}
with open(meme_file) as handle:
    motif_list = motifs.parse(handle, "pfm-four-columns")
    i = 0
    for motif in motif_list:
        motif.name = motif_names[i]
        hocomoco_motif_dict[motif.name] = motif
        i += 1


def reverse_complement(seq):
    complement = str.maketrans("ACGTacgt", "TGCAtgca")
    return seq.translate(complement)[::-1]


def local_bitscore_tfbs(motif, sequence, motif_start, motif_end, variant_pos, strand):
    """
    Calculate the local bitscore for a given motif and sequence, using a window of 5 bases around the variant position.
    """
    zero_based_motif_start = motif_start - 1  # start should be 0-based and end 1-based
    pwm = motif.pwm
    zero_based_variant_pos = variant_pos - 1

    # Get only subset of sequence which is overlapped with the motif
    sequence_of_interest_start = max(zero_based_motif_start, zero_based_variant_pos - 5)
    sequence_of_interest_end = min(motif_end, zero_based_variant_pos + 5)

    motif_related_start = max(0, sequence_of_interest_start - zero_based_motif_start)
    motif_related_end = min(motif_end - zero_based_motif_start, sequence_of_interest_end - zero_based_motif_start)

    # Normalize the score for the length of the motif
    additional_score = 10 - (motif_related_end - motif_related_start)
    motif_sequence = sequence[zero_based_motif_start:motif_end]
    
    # Use the reverse complement sequence if the hit is on the "-" strand
    if strand == "-":
        motif_sequence = reverse_complement(motif_sequence)
    
    motif_part = motif_sequence[motif_related_start:motif_related_end]
    
    # Crop the PWM to the motif-related part
    modified_pwm = {nuc: scores[motif_related_start:motif_related_end] for nuc, scores in pwm.items()}

    return additional_score + round(sum(modified_pwm[nuc][i] for i, nuc in enumerate(motif_part)), 4)

Calculate a bitscore for each motif match in the context of the variant position. 🧮 We extract a sequence around the variant position with the same length as the motif and calculate the bitscore of that sequence with the motif PWM. 📊

In [ ]:
variant_overlapping_tfbs['REF_bit_score'] = variant_overlapping_tfbs.apply(
    lambda row: local_bitscore_tfbs(
        motif=hocomoco_motif_dict[row["motif_id"]],
        sequence=row['ref_sequence'],
        motif_start=row['start'],
        motif_end=row['stop'],
        variant_pos=row['variant_pos'],
        strand=row['strand']
    ),
    axis=1
)
variant_overlapping_tfbs['ALT_bit_score'] = variant_overlapping_tfbs.apply(
    lambda row: local_bitscore_tfbs(
        motif=hocomoco_motif_dict[row["motif_id"]],
        sequence=row['alt_sequence'],
        motif_start=row['start'],
        motif_end=row['stop'],
        variant_pos=row['variant_pos'],
        strand=row['strand']
    ),
    axis=1
)

variant_overlapping_tfbs['max_bit_score'] = variant_overlapping_tfbs[['REF_bit_score', 'ALT_bit_score']].max(axis=1).round(3)

> **✅ Result:**  
> The bitscore now correctly ranks motif fits. 🎯  
> - **Higher bitscore = better motif fit** 📈
> - Filter for a minimal bitscore to avoid weak matches ⚠️

In [ ]:
interesting_columns =["variant_id", "motif_id", "clinical_significance", "max_bit_score"]

variant_overlapping_tfbs.sort_values(by='max_bit_score', ascending=False).drop_duplicates(subset='variant_id', keep='first')[interesting_columns].head(10)

Let's look at the highest difference between reference and alternative allele bitscores: 📊🔍

In [ ]:
interesting_columns = ["variant_id", "motif_id", "clinical_significance", "REF_bit_score", "ALT_bit_score", "delta_bit_score"]

variant_overlapping_tfbs['delta_bit_score'] = (
    variant_overlapping_tfbs['ALT_bit_score'] - variant_overlapping_tfbs['REF_bit_score']
).round(3).abs()

variant_overlapping_tfbs.sort_values(by='delta_bit_score', ascending=False).drop_duplicates(
    subset='variant_id', keep='first'
)[interesting_columns].head(10)

Does this result make sense? 🤔 For example, the variant **5-1295135-G-A** creates a strong match for multiple motifs like ELK4 and GABPA in the alternative allele. See here: 👇

In [ ]:
variant_overlapping_tfbs[variant_overlapping_tfbs['variant_id'] == "5-1295135-G-A"].sort_values(
    by='delta_bit_score', ascending=False
)[interesting_columns]

Do we have more functional evidence? **YES!** 🎉 Saturation mutagenesis experiments (so-called Massively Parallel Reporter Assays, MPRAs) have shown that this variant creates a new binding site for GABPA, which leads to increased TERT expression and is linked to cancer. 🔬📈

Please go to https://kircherlab.bihealth.org/satMutMPRA/ → **Promoter: TERT**, **Variant plot** and locate your variants. What do you see? 👀

**🤔 BTW, what is an MPRA?** That you will learn next in `Epigenetic Data and Regulatory Codes: Machine Learning Perspectives` 🚀